# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os, warnings

warnings.filterwarnings('ignore')

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_dataDF = pd.read_csv(output_data_file)
weather_dataDF.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pevek,69.7008,170.3133,36.07,84,96,4.23,RU,1658616880
1,Port Elizabeth,-33.9180,25.5701,53.89,86,40,13.80,ZA,1658617000
2,Mataura,-46.1927,168.8643,38.48,69,80,1.36,NZ,1658616890
3,Alofi,-19.0595,-169.9187,74.43,69,89,20.04,NU,1658617231
4,Saint-Pierre,-21.3393,55.4781,67.68,76,100,5.75,RE,1658617231
5,Cape Town,-33.9258,18.4232,55.38,72,20,10.36,ZA,1658616848
6,Talara,-4.5772,-81.2719,65.21,85,8,14.29,PE,1658617232
7,Tiksi,71.6872,128.8694,48.85,78,31,6.17,RU,1658616981
8,Umm Hājar,13.2954,19.6966,84.78,56,95,8.66,TD,1658617232
9,Yellowknife,62.4560,-114.3525,72.12,37,75,19.57,CA,1658617001


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
weather_dataDF.count()


City          570
Lat           570
Lng           570
Max Temp      570
Humidity      570
Cloudiness    570
Wind Speed    570
Country       567
Date          570
dtype: int64

In [6]:
# Store 'Lat' and 'Lng' into  locations and store humidity 
locations = weather_dataDF[["Lat", "Lng"]].astype(float)
humidity = weather_dataDF["Humidity"].astype(float)
maxHumidity = humidity.max()

# HINT: be sure to handle NaN values
weather_dataDF = weather_dataDF.dropna()
#weather_dataDF.count()

figure_layout = {
    'width': '800px',
    'height': '420px'
}

fig = gmaps.figure(layout=figure_layout, center = [0,0], zoom_level =2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity= maxHumidity,point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
filtered_weatherDF = weather_dataDF.loc[(weather_dataDF["Max Temp"] > 70) & (weather_dataDF["Max Temp"] < 80) & 
                                        (weather_dataDF["Cloudiness"] == 0) & (weather_dataDF["Wind Speed"] < 10),:]

filtered_weatherDF.reset_index(inplace=True)
del filtered_weatherDF['index']
filtered_weatherDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,72.91,63,0,8.39,PF,1658616783
1,Birjand,32.8663,59.2211,74.53,44,0,9.60,IR,1658617263
2,Erzin,36.9559,36.2000,74.66,63,0,2.71,TR,1658617142
3,Mahon,39.8885,4.2658,79.36,83,0,3.44,ES,1658617156
4,Custoias,41.1053,-7.3210,73.09,43,0,5.23,PT,1658617301
5,Mogadouro,41.3403,-6.7119,75.36,34,0,3.56,PT,1658617217
6,Kiryat Gat,31.6100,34.7642,74.71,79,0,1.16,IL,1658616999
7,Tarsus,36.9188,34.8784,77.50,54,0,4.41,TR,1658617373
8,Caravelas,-17.7125,-39.2481,70.59,76,0,6.64,BR,1658617131
9,Nouadhibou,20.9310,-17.0347,76.98,78,0,5.75,MR,1658616914


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#making a copy of filtered_weatherDF
hotel_df = filtered_weatherDF

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
index = 0

for i, row in hotel_df.iterrows():
    
    city = row["City"]
    print(city)
    Lat = hotel_df.loc[i]["Lat"]
    Lng = hotel_df.loc[i]["Lng"]
    
    params = {
        "location": f"{Lat}, {Lng}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key          
    }
    
    print(f"Retrieving Results for Index {i}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[i, 'Hotel Name'] = results[0]['name']
                        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------")

Rikitea
Retrieving Results for Index 0: Rikitea.
Closest hotel is People ThankYou.
--------------------
Birjand
Retrieving Results for Index 1: Birjand.
Closest hotel is Birjand Tourist Hotel.
--------------------
Erzin
Retrieving Results for Index 2: Erzin.
Closest hotel is Artemis Otel.
--------------------
Mahon
Retrieving Results for Index 3: Mahon.
Closest hotel is ARTIEM Capri.
--------------------
Custoias
Retrieving Results for Index 4: Custoias.
Closest hotel is Casa Do Adro / Solar Corte Real - José Carlos Pereira Corte Real.
--------------------
Mogadouro
Retrieving Results for Index 5: Mogadouro.
Closest hotel is Pensão Russo.
--------------------
Kiryat Gat
Retrieving Results for Index 6: Kiryat Gat.
Closest hotel is OlusHome.
--------------------
Tarsus
Retrieving Results for Index 7: Tarsus.
Closest hotel is Osmanli Marco Pasha Hotel.
--------------------
Caravelas
Retrieving Results for Index 8: Caravelas.
Closest hotel is Pousada dos Navegantes.
--------------------
No

In [9]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Rikitea,-23.1203,-134.9692,72.91,63,0,8.39,PF,1658616783,People ThankYou
1,Birjand,32.8663,59.2211,74.53,44,0,9.60,IR,1658617263,Birjand Tourist Hotel
2,Erzin,36.9559,36.2000,74.66,63,0,2.71,TR,1658617142,Artemis Otel
3,Mahon,39.8885,4.2658,79.36,83,0,3.44,ES,1658617156,ARTIEM Capri
4,Custoias,41.1053,-7.3210,73.09,43,0,5.23,PT,1658617301,Casa Do Adro / Solar Corte Real - José Carlos ...
5,Mogadouro,41.3403,-6.7119,75.36,34,0,3.56,PT,1658617217,Pensão Russo
6,Kiryat Gat,31.6100,34.7642,74.71,79,0,1.16,IL,1658616999,OlusHome
7,Tarsus,36.9188,34.8784,77.50,54,0,4.41,TR,1658617373,Osmanli Marco Pasha Hotel
8,Caravelas,-17.7125,-39.2481,70.59,76,0,6.64,BR,1658617131,Pousada dos Navegantes
9,Nouadhibou,20.9310,-17.0347,76.98,78,0,5.75,MR,1658616914,El Medina


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
# Customize the size of the figure

figure_layout = {
    'width': '600px',
    'height': '420px'   
}

fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px', width='600px'))